This script is to get the activation thresholds for various carrier frequencies and depths of modulation 

In [2]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import math
import os, sys
#Add main file to path
sys.path.insert(1,r'PATH WHERE PROCESSING_FUNCTIONS IS SAVED')

#import commonly used processing functions 
import processing_functions as process
from scipy.io import savemat

In [7]:
#carrier frequency selected
Frequencies=[0, 500, 2000,5000,10000]
Frequencies=[0]

paths=[]
paths.append(r'PATH WHERE THE HDF5 FILES ARE SAVED')
include_endings=1
include_nerve=1

for path in paths:
    mdic={}
    mdic['Frequencies']=Frequencies
    Current_density_max=[]
    Current_density_sum=[]
    Current_density_av=[]
    for F in Frequencies:
        open_this=os.path.join(path,(str(F) +".hdf5"))
        f=h5py.File(open_this,"r")
        endings_activation=[]
        axons_activation=[]
        if include_endings==1:
            folder=f.get('Endings Potential')
            for ending in range(189):
                ending_name='ending_'+str(ending)+'_neuron'
                ending_folder=folder.get(ending_name)
                nodes=ending_folder.keys()
                last_node=0
                for node in nodes:
                    node_number=process.get_node(node)
                    if node_number>last_node:
                        last_node=node_number

                node_before_last=str(last_node-1)
                saved_node=next((s for s in nodes if node_before_last in s), None)
                ending_activation=ending_folder.get(saved_node)
                endings_activation.append(ending_activation[1,:])
            endings_name='Endings_activation_'+str(F)
            mdic[endings_name]=endings_activation

        if include_nerve==1:
            folder=f.get('Nerve Membrane Potential')
            for axon in range(100):
                axon_name='Median_'+str(axon)+'_neuron'
                axon_folder=folder.get(axon_name)
                nodes=axon_folder.keys()
                last_node=0
                for node in nodes:
                    node_number=process.get_node(node)
                    if node_number>last_node:
                        last_node=node_number
                node_before_last=str(last_node-1)
                node_before_last=str(1)
                saved_node=next((s for s in nodes if node_before_last in s), None)
                axon_activation=axon_folder.get(saved_node)
                axons_activation.append(axon_activation[1,:])
            nerve_name='Nerve_activation_'+str(F)
            mdic[nerve_name]=axons_activation

        #Get the current density
        folder=f.get('Current Density')
        all_vals=folder.get(str(F))

        #Remove rows with nan
        indexList = [np.all(i) for i in np.isnan(all_vals)]
        all_vals = np.delete(all_vals, indexList, axis=0)
        max_val,sum_vals,average_val=process.Current_density(all_vals)
        Current_density_max.append(max_val)
        Current_density_sum.append(sum_vals)
        Current_density_av.append(average_val)
        
        f.close()

    mdic['Max_current_density']=Current_density_max
    mdic['Sum_current_density']=Current_density_sum
    file_name='Comfort_metrics.mat'
    savemat(os.path.join(path,file_name),mdic)
